In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from dotenv import dotenv_values
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [2]:
config = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

In [3]:
labels_to_text = {
    "datePublished": "date published",
    "isbn": "isbn",
    "numberOfPages": "number of pages",
    "worstRating": "worst rating",
    "priceCurrency": "price currency",
    "publisher": "publisher",
    "author": "author",
    "bookFormat": "book format",
    "inLanguage": "in language",
    "ratingValue": "rating value",
    "description": "description",
    "price": "price",
    "image": "image",
    "url": "url",
    "availability": "availability",
    "genre": "genre",
    "bestRating": "best rating",
    "itemCondition": "item condition",
    "review": "review",
    "startDate": "start date",
    "endDate": "end date",
    "location": "location",
    "validFrom": "valid from",
    "eventStatus": "event status",
    "eventAttendanceMode": "event attendance mode",
    "organizer": "organizer",
    "category": "category",
    "validThrough": "valid through",
    "telephone": "telephone",
    "duration": "duration",
    "email": "email",
    "streetAddress": "street address",
    "addressRegion": "region of address",
    "addressLocality": "locality of address",
    "priceRange": "price range",
    "postalCode": "postal code",
    "addressCountry": "address country",
    "faxNumber": "fax number",
    "dateCreated": "date created",
    "director": "director",
    "contentRating": "content rating",
    "actor": "actor",
    "inAlbum": "in album",
    "byArtist": "by artist",
    "jobTitle": "job title",
    "birthDate": "birth date",
    "gender": "gender",
    "nationality": "nationality",
    "weight": "weight",
    "releaseDate": "release date",
    "copyrightYear": "copyright year",
    "bookEdition": "book edition",
    "headline": "headline",
    "text": "text",
    "address": "address",
    "performer": "performer",
    "longitude": "longitude",
    "latitude": "latitude",
    "currenciesAccepted": "currencies accepted",
    "checkoutTime": "checkout time",
    "checkInTime": "check-in time",
    "amenityFeature": "amenity feature",
    "paymentAccepted": "payment accepted",
    "availableLanguage": "available language",
    "brand": "brand",
    "openingHours": "opening hours",
    "datePosted": "date posted",
    "employmentType": "employment type",
    "hiringOrganization": "hiring organization",
    "opens": "opens",
    "dayOfWeek": "day of week",
    "closes": "closes",
    "productionCompany": "production company",
    "countryOfOrigin": "country of origin",
    "numTracks": "number of tracks",
    "track": "track",
    "givenName": "given name",
    "familyName": "family name",
    "birthPlace": "birth place",
    "honorificSuffix": "honorific suffix",
    "alumniOf": "alumni of",
    "deathDate": "death date",
    "measurements": "measurements",
    "unitCode": "unit code",
    "productID": "product ID",
    "unitText": "unit text",
    "availableDeliveryMethod": "available delivery method",
    "model": "model",
    "manufacturer": "manufacturer",
    "color": "color",
    "gtin": "gtin",
    "material": "material",
    "servingSize": "serving size",
    "recipeInstructions": "recipe instructions",
    "recipeIngredient": "recipe ingredient",
    "cookTime": "cook time",
    "prepTime": "prep time",
    "totalTime": "total time",
    "nutrition": "nutrition",
    "recipeCategory": "recipe category",
    "recipeCuisine": "recipe cuisine",
    "cookingMethod": "cooking method",
    "suitableForDiet": "suitable for diet",
    "servesCuisine": "serves cuisine",
    "awayTeam": "away team",
    "homeTeam": "home team",
    "partOfSeries": "part of series",
    "episodeNumber": "episode number"
}

In [4]:
len(labels_to_text)

108

In [4]:
text_to_label = {
    "date published": "datePublished",
    "isbn": "isbn",
    "number of pages": "numberOfPages",
    "worst rating": "worstRating",
    "price currency": "priceCurrency",
    "publisher": "publisher",
    "author": "author",
    "book format": "bookFormat",
    "in language": "inLanguage",
    "rating value": "ratingValue",
    "description": "description",
    "price": "priceRange",
    "image": "image",
    "url": "url",
    "availability": "availability",
    "genre": "genre",
    "best rating": "bestRating",
    "item condition": "itemCondition",
    "review": "review",
    "start date": "startDate",
    "end date": "endDate",
    "location": "location",
    "valid from": "validFrom",
    "event status": "eventStatus",
    "event attendance mode": "eventAttendanceMode",
    "organizer": "organizer",
    "category": "category",
    "valid through": "validThrough",
    "telephone": "telephone",
    "duration": "duration",
    "email": "email",
    "street address": "streetAddress",
    "region of address": "addressRegion",
    "locality of address": "addressLocality",
    "price range": "priceRange",
    "postal code": "postalCode",
    "address country": "addressCountry",
    "fax number": "faxNumber",
    "date created": "dateCreated",
    "director": "director",
    "content rating": "contentRating",
    "actor": "actor",
    "in album": "inAlbum",
    "by artist": "byArtist",
    "job title": "jobTitle",
    "birth date": "birthDate",
    "gender": "gender",
    "nationality": "nationality",
    "weight": "weight",
    "release date": "releaseDate",
    "copyright year": "copyrightYear",
    "book edition": "bookEdition",
    "headline": "headline",
    "text": "text",
    "address": "address",
    "performer": "performer",
    "longitude": "longitude",
    "latitude": "latitude",
    "currencies accepted": "currenciesAccepted",
    "checkout time": "checkoutTime",
    "check-in time": "checkInTime",
    "amenity feature": "amenityFeature",
    "payment accepted": "paymentAccepted",
    "available language": "availableLanguage",
    "brand": "brand",
    "opening hours": "openingHours",
    "date posted": "datePosted",
    "employment type": "employmentType",
    "hiring organization": "hiringOrganization",
    "opens": "opens",
    "day of week": "dayOfWeek",
    "closes": "closes",
    "production company": "productionCompany",
    "country of origin": "countryOfOrigin",
    "number of tracks": "numTracks",
    "track": "track",
    "given name": "givenName",
    "family name": "familyName",
    "birth place": "birthPlace",
    "honorific suffix": "honorificSuffix",
    "alumni of": "alumniOf",
    "death date": "deathDate",
    "measurements": "measurements",
    "unit code": "unitCode",
    "product ID": "productID",
    "unit text": "unitText",
    "available delivery method": "availableDeliveryMethod",
    "model": "model",
    "manufacturer": "manufacturer",
    "color": "color",
    "gtin": "gtin",
    "material": "material",
    "serving size": "servingSize",
    "recipe instructions": "recipeInstructions",
    "recipe ingredient": "recipeIngredient",
    "cook time": "cookTime",
    "prep time": "prepTime",
    "total time": "totalTime",
    "nutrition": "nutrition",
    "recipe category": "recipeCategory",
    "recipe cuisine": "recipeCuisine",
    "cooking method": "cookingMethod",
    "suitable for diet": "suitableForDiet",
    "serves cuisine": "servesCuisine",
    "away team": "awayTeam",
    "home team": "homeTeam",
    "part of series": "partOfSeries",
    "episode number": "episodeNumber"
}

## Load test (and training) set

In [5]:
with open('sotabv2-cpa-train-column.pkl', "rb") as f:
    train = pickle.load(f)
with open('sotabv2-cpa-sample-test-column.pkl', "rb") as f:
    test = pickle.load(f) 
examples = [example[2] for example in test ]     
labels = [example[3] for example in test ]

train_examples = [ example[2] for example in train ]
train_labels = [ labels_to_text[example[3]] for example in train ]

In [7]:
len(train)

109994

In [8]:
len(test)

509

In [6]:
labels_joined = ", ".join([labels_to_text[label] for label in list(set(labels))])   
print(labels_joined)

total time, currencies accepted, date posted, country of origin, review, alumni of, organizer, given name, number of pages, item condition, duration, hiring organization, employment type, email, director, date published, weight, text, address country, valid through, family name, away team, author, copyright year, amenity feature, gtin, serving size, valid from, recipe cuisine, recipe instructions, home team, recipe category, by artist, nutrition, birth place, prep time, birth date, production company, worst rating, day of week, checkout time, available delivery method, headline, publisher, opening hours, number of tracks, event status, locality of address, opens, in album, measurements, content rating, material, end date, location, region of address, honorific suffix, postal code, book edition, part of series, telephone, availability, best rating, address, closes, job title, image, nationality, gender, product ID, rating value, track, available language, price currency, death date, coo

In [7]:
model_name = 'gpt-3.5-turbo-1106'
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model=model_name, max_tokens=4)   #max_tokens=4

## Choose setup: zero-shot, one-shot or five-shot

CPA COLUMN


ZERO-SHOT

In [ ]:
#role
nr="zero"
prompt_name = "r"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))

    messages.append(HumanMessage(content=f"Classify this relation: {example}"))

    print(messages)
    res = chat(messages)
    preds.append(res.content)

In [37]:
#role + instructions 
nr="zero"
prompt_name = "r+i"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))

    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [51]:
#role + instructions + step by step
nr="zero"
prompt_name = "r+i+s_b_s"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Let's think step by step. Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))

    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [68]:
#role + instructions + motivation
nr="zero"
prompt_name = "r+i+m"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))

    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [86]:
#role + instructions (motivation as instruction)
nr="zero"
prompt_name = "r+i(m)"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation! 6. Your answer is very important. Take your time and think well before answering!"))
 
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + CONTEXT
nr="zero"
prompt_name = "r+i+c"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content=f"CONTEXT: Column Property Annotation is a sub-task of Table Annotation and refers to predicting the semantic relation between two or more columns. You have the same task, you are required to annotate the relation between two given columns."))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

ONE-SHOT

In [150]:
#role
import random

nr="one"
prompt_name = "r"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [262]:
#role
import random

nr="one"
prompt_name = "r2"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions
import random 
 
nr="one"
prompt_name = "r+i"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions  P2
import random 
 
nr="one"
prompt_name = "r+i2"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + step by step
import random

nr="one"
prompt_name = "r+i+s_b_s"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Let's think step by step. Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + motivation
import random

nr="one"
prompt_name = "r+i+m"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))

    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + CONTEXT
import random

nr="one"
prompt_name = "r+i+c"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between two columns with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between these two columns. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content=f"CONTEXT: Column Property Annotation is a sub-task of Table Annotation and refers to predicting the semantic relation between two or more columns. You have the same task, you are required to annotate the relation between two given columns."))
    
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [10]:
#role + instructions + CONTEXT example
import random

nr="one"
prompt_name = "r2+i2+c.example"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content=f"CONTEXT: Column Property Annotation is a sub-task of Table Annotation and refers to predicting the semantic relation between two or more columns. You have the same task, here is an example how you could solve a CPA task: 'Classify the relationship between these two columns: Columm1: Dog, Cat, Dog.  Column2: lis, moli, brauni.'"
                                  "First we check Columm1: Dog, Cat, Dog."   
                                  "Now we check Column2: lis, moli, brauni. Analyze Column 2 in relation to Column 1. Predict the relation between Column 2 and Column 1"
                                  "Answer: Column 2: pet names"))
    
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + CONTEXT test
import random

nr="one"
prompt_name = "r2+i2+c.test"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
  
    messages.append(SystemMessage(content=f"CONTEXT: Classify this relation: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

FIVE-SHOT

In [23]:
#role
import random

nr= "five"
prompt_name = "r2"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))

    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [41]:
#role + instructions
import random 
 
nr= "five"
prompt_name = "r2+i2"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [55]:
#role + instructions + step by step
import random

nr= "five"
prompt_name = "r2+i2+s_b_s"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Let's think step by step. Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [75]:
#role + instructions + motivation
import random

nr= "five"
prompt_name = "r2+i2+m"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))

    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [92]:
#role + instructions + CONTEXT
import random

nr= "five"
prompt_name = "r2+i2+c"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great Table Annotation Specialist and your task is to classify the relationship between Column 1 and Column 2 with ONLY ONE of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Review the provided column values. 2. Carefully examine the values of the two columns. 3. Select a single label that best represents the relationship between Column 2 with Column 1. 4. Answer with your final selected label. 5. Ensure that you answer with ONLY ONE label from the provided label-set for each relation!"))
    messages.append(SystemMessage(content=f"CONTEXT: Column Property Annotation is a sub-task of Table Annotation and refers to predicting the semantic relation between two or more columns. You have the same task, you are required to annotate the relation between two given columns."))
    
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this relation: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this relation: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
preds

In [96]:
file_name=f'Predictions/{model_name}/column/{nr}-shot/chat-column-{prompt_name}-{nr}-shot.pkl'
f = open(file_name,'wb')
pickle.dump(preds,f)
f.close()

## Evaluation

In [ ]:
predictions = []
for i, pred in enumerate(preds):
    from_sent = re.findall('"([^"]*)"',pred)
    if len(from_sent) == 0:
        if ":" in pred:
            pred = pred.split(':')[1]
        if "." in pred:
            pred = pred.split('.')[0]
        pred = pred.strip()
        
        if pred in text_to_label:
            predictions.append(text_to_label[pred])
        else:
            if any(label in pred for label in text_to_label):
                for label in text_to_label:
                    if label in pred:
                        predictions.append(text_to_label[label])
                        break
            else:
                print(f"For test example {i} out of label space prediction: {pred}")
                predictions.append('-')
    else:
        if from_sent[0] in text_to_label:
            predictions.append(text_to_label[from_sent[0]])
        else:
            print(f"For test example {i} out of label space prediction: {pred}")
            predictions.append('-')

In [ ]:
predictions

### Calculate Precision, Recall, Macro-F1 and Micro-F1

In [21]:
def calculate_f1_scores(y_tests, y_preds, num_classes, types):

    y_tests = [types.index(y) for y in y_tests]
    y_preds = [types.index(y) for y in y_preds]
    
  
    cm = np.zeros(shape=(num_classes,num_classes))
    
    for i in range(len(y_tests)):
        cm[y_preds[i]][y_tests[i]] += 1
        
    report = {}
    
    for j in range(len(cm[0])):
        report[j] = {}
        report[j]['FN'] = 0
        report[j]['FP'] = 0
        report[j]['TP'] = cm[j][j]

        for i in range(len(cm)):
            if i != j:
                report[j]['FN'] += cm[i][j]
        for k in range(len(cm[0])):
            if k != j:
                report[j]['FP'] += cm[j][k]

        precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
        recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])
        f1 = 2*precision*recall / (precision + recall)
        
        if np.isnan(f1):
            f1 = 0
        if np.isnan(precision):
            f1 = 0
        if np.isnan(recall):
            f1 = 0

        report[j]['p'] =  precision
        report[j]['r'] =  recall
        report[j]['f1'] = f1
    
    all_fn = 0
    all_tp = 0
    all_fp = 0

    for r in report:
        if r != num_classes-1:
            all_fn += report[r]['FN']
            all_tp += report[r]['TP']
            all_fp += report[r]['FP']
        
    class_f1s = [ report[class_]['f1'] for class_ in report]
    class_p = [ 0 if np.isnan(report[class_]['p']) else report[class_]['p'] for class_ in report]
    class_r = [ 0 if np.isnan(report[class_]['r']) else report[class_]['r'] for class_ in report]
    macro_f1 = sum(class_f1s[:-1]) / (num_classes-1)
    
    p =  sum(class_p[:-1]) / (num_classes-1)
    r =  sum(class_r[:-1]) / (num_classes-1)
    micro_f1 = all_tp / ( all_tp + (1/2 * (all_fp + all_fn) )) 
    
    per_class_eval = {}
    for index, t in enumerate(types[:-1]):
        per_class_eval[t] = {"Precision":class_p[index], "Recall": class_r[index], "F1": class_f1s[index]}
    
    evaluation = {
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Precision": p,
        "Recall": r
    }
    
    return [ evaluation, per_class_eval]


In [ ]:
types = list(set(labels))
types = types + ["-"] if "-" in predictions else types
evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)

In [ ]:
evaluation

In [ ]:
per_class_eval

## Error Analysis

In [ ]:
errors = 0
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        errors += 1
        print(f"Predicted as {predictions[i]} when it was {labels[i]}")
errors

### Re-load previous preds files

In [110]:
with open(f'Predictions/{model_name}/column/{nr}-shot/chat-column-{prompt_name}-{nr}-shot.pkl', "rb") as f:
    preds = pickle.load(f)

In [ ]:
preds